**Niks Pille**

All Tickers

In [9]:
#Getting all tickers in a list
All_Tickers = []
for i in range(4521):
    All_Tickers.append(Only_Common_Stocks['symbol'][i])

Creating 'Stock_With_All_Data'

In [171]:
Stocks_With_All_Data = []
for i in range(4521):
    if len(yf.Ticker(All_Tickers[i]).history(start='2000-01-02', end='2020-01-01')[['Close']]) == 5031:
        Stocks_With_All_Data.append(All_Tickers[i])

- AGM.A: No data found, symbol may be delisted
- ALUS: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- AMHC: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- ARQT: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BATL: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BDTX: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BEAM: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BF.A: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted
- BH.A: No data found, symbol may be delisted
- BIO.B: No data found for this date range, symbol may be delisted
- BRK.A: No data found, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
- BSBK: Data doesn't exist for startDate = 946767600, endDate = 1577833200
- BWL.A: No data found, symbol may be delisted
- CHPM: Data doesn't exist for startDate = 946767600, endDate 

In [186]:
with open('Stocks_With_Data.csv','w',newline='') as file:
    wr = csv.writer(file, quoting=csv.QUOTE_ALL)
    wr.writerow(Stocks_With_All_Data)

Creating 'Three_Month_Treasury_Bill'

In [38]:
#DTB3 is taken from https://fred.stlouisfed.org/
DTB3 = pd.read_csv('DTB3.csv', index_col = 0)
#Using Apple as a benchmark
aapl = yf.Ticker('aapl').history(start='2000-01-02', end='2020-01-01')[['Close']]
#Getting dates from Apple
Dates = []
for i in range(len(aapl)):
    if str(aapl.index[i])[:10] in DTB3.index:
        Dates.append(str(aapl.index[i])[:10])
#Sorting out missing values
Three_Month_Treasury_Bill = DTB3.loc[Dates][DTB3.loc[Dates].DTB3 != '.']
#Creating a csv-file with the Three Month Treasury Bill
Three_Month_Treasury_Bill.to_csv('Three_Month_Treasury_Bill.csv')

Creating 'All_Stocks'

In [45]:
#Make a table with the new dates
Ny = DTB3.loc[Dates]
#Make a new column with strings
Ny['Datoer'] = Dates
#Set the new column as index to make it a callable string
Nyy = Ny.set_index('Datoer')
#A for loop to get all tables for the chosen stocks into a single list, to be used in concatenating
stock = []
for i in list(dict.fromkeys(Stocks)):
    asset = yf.Ticker(i).history(start='2000-01-02', end='2020-01-01')[['Close']]
    asset.columns = [i]
    stock.append(asset)
#Concat all the tables
Concat_Table = pd.concat([i for i in stock], axis=1, join='inner')
#Concat the stocks with the risk free rate
All_Tables = pd.concat([Nyy, Concat_Table.reindex(Nyy.index)],axis=1)
#Save to a csv file
All_Tables.to_csv('All_Stocks.csv')

Creating 'Table'

In [48]:
Everything = pd.read_csv('All_Stocks.csv', index_col=0)
#Russell3000 is used as the market rate since it covers 98% of the American market
Russell_3000 = yf.Ticker('^RUA').history(start='2000-01-02', end='2020-01-01')[['Close']]
#Sorting out missing values from the risk-free rate
gj = Everything.loc[:][Everything.loc[:].DTB3 != '.']
#Making a new date list with the dates from Russell 3000
Datess = []
for i in range(len(Russell_3000)):
    if str(Russell_3000.index[i])[:10] in gj.index:
        Datess.append(str(Russell_3000.index[i])[:10])
#Making the dates strings so they are callable
gh = []
for i in range(len(Russell_3000)):
    gh.append(str(Russell_3000.index[i])[:10])
#Making a new column with the string dates and set it as index
Russell_3000['Datoer'] = gh
gk = Russell_3000.set_index('Datoer')
#Set the name of the column
gk.columns = ['Russell3000']
#Concat Russell3000 to the rest
Table = pd.concat([gj.loc[Datess],gk.loc[Datess]],axis=1)
Table.to_csv('Table.csv')

# Bachelorproject

Documentation:  
https://addisonlynch.github.io/iexfinance/stable/stocks.html  
https://github.com/ranaroussi/yfinance

## Getting Data

In [1]:
#All packages used
from iexfinance.stocks import Stock
from iexfinance.refdata import get_symbols
import yfinance as yf
import numpy as np
import pandas as pd
import csv
token = 'sk_32c5d1b5f6b044f294391ae88eac3e47'

In [2]:
#Get a table with everything on the financial markets in USA
Alle_symboler = get_symbols(output_format='pandas', token=token)
#Sorting out all the non common stocks
Only_Common_Stocks = get_symbols()[get_symbols().type == 'cs'].reset_index(drop=True)

In [3]:
with open('Stocks_With_Data.csv',newline='') as file:
    reader = csv.reader(file)
    Stocks_With_All_Data = list(reader)
#Drawing 300 random stocks for which to be analyzed
Stocks = []
np.random.seed(2020)
for i in range(300):
    Stocks.append(Stocks_With_All_Data[0][np.random.randint(0,len(Stocks_With_All_Data[0]))])

Stocks.sort()
Every_Stock = list(dict.fromkeys(Stocks))

### Adjusting so all data match with no missing values

In [4]:
All = pd.read_csv('Table.csv', index_col=0)

In [15]:
columns = []
for i in range(283):
    columns.append(All.columns[i])
#Creating a table with the log values
Logtable = pd.DataFrame()
for i in range(283):
    Logtable[columns[i]] = All[columns[i]].apply(np.log)

In [84]:
Daily = pd.DataFrame()
for i in range(len(columns)):
    Daily[columns[i]] = np.diff(Logtable[columns[i]],axis=0)

In [85]:
Daily_returns = Daily.set_index(Logtable.index.values[0:-1])
Daily_returns.to_csv('Daily_returns.csv')

In [7]:
Daily_returns = pd.read_csv('Daily_returns.csv', index_col=0)

In [17]:
Excess_return = pd.DataFrame()

for i in range(len(columns)):
    Excess_return[columns[i]] = Daily_returns[columns[i]] - Daily_returns['DTB3']
del Excess_return['DTB3']

In [35]:
Excess_return.to_csv('Excess_return.csv')

In [36]:
Excess_return

,A,AB,ABC,ADM,AEE,AEM,AES,AGM,AIT,AJG,...,WTM,WVVI,WWD,WWE,WYY,XLNX,XRAY,XRX,Y,Russell3000
2000-01-03,-0.079649,-0.004035,-0.072571,-0.011364,0.000000,-0.026404,-0.040341,-0.026170,-0.015306,-0.046030,...,-0.008499,0.030772,0.002886,-0.043394,0.000000,-0.022990,0.000000,-0.047507,-0.005382,-0.039434
2000-01-04,-0.065833,0.002139,0.074003,-0.017735,0.035907,-0.010292,0.008614,-0.056384,-0.025304,0.002134,...,0.026620,0.027957,-0.028175,0.034625,-0.015523,-0.024174,0.016584,0.048270,-0.005595,-0.000190
2000-01-05,-0.032993,0.021677,0.079331,0.011487,0.002467,0.022421,0.013955,0.043379,-0.012895,0.003013,...,-0.001610,-0.024155,-0.006207,-0.059853,0.192597,-0.084961,0.005698,-0.015190,0.011100,0.005222
2000-01-06,0.085669,0.038222,0.127620,0.021479,0.020189,0.031919,0.024853,0.032338,-0.026970,0.043969,...,-0.000603,0.035584,0.005731,0.010139,0.005731,0.113145,0.004321,0.029081,0.032587,0.032486
2000-01-07,0.055024,-0.021889,0.020469,-0.003824,-0.011027,0.005822,0.040094,0.002905,0.020801,-0.040719,...,-0.005882,-0.033677,-0.065572,-0.006761,-0.099134,0.045698,0.018492,-0.027174,-0.019333,0.011535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-23,0.006901,0.007791,0.004434,0.007744,0.007893,0.032966,0.003377,-0.000174,0.011708,0.009622,...,0.011805,0.007863,0.002142,0.002422,-0.018887,0.002258,0.006956,0.006970,0.002668,0.006516
2019-12-24,0.001761,0.002715,-0.004360,0.002621,-0.001462,-0.000491,0.008627,0.006365,0.002103,-0.000106,...,-0.012207,0.005706,-0.002447,0.008003,-0.025975,0.001936,-0.010208,0.002959,0.002836,0.004546
2019-12-26,0.006121,-0.002719,0.004818,0.008652,0.015213,0.006309,0.008996,0.000469,0.001207,0.008595,...,-0.000243,0.006473,0.004612,-0.001530,0.057766,0.003823,0.011941,0.001896,0.007669,0.005946
2019-12-27,-0.003997,0.009869,-0.015136,-0.003926,0.003422,0.018980,0.003020,-0.002895,0.003914,0.001589,...,0.018422,-0.007138,-0.001864,-0.011500,-0.025318,-0.007478,-0.007771,-0.004327,0.009151,-0.005592
